In [5]:
import os
from PIL import Image
import util.image_import as ii
from glob import glob
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from keras import backend as K
import matplotlib.pyplot as plt
from keras.applications import mobilenet
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.models import Model, Sequential
from keras import models
from keras import layers
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam, RMSprop, SGD
from util.plots import plot_history
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Trick to configure gpu memory dynamically

In [6]:
cfg = K.tf.ConfigProto() # Config to dynamically add memory
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))# set this TensorFlow session as the default session for Keras.

### Subtracting mean and normalizing to save pictures

In [7]:
# Importing the images
train_pot = glob(os.path.join("../dataset2/res_still/train/potato", "*.jpg"))
train_cat = glob(os.path.join("../dataset2/res_still/train/catfood", "*.jpg"))
train_tab = glob(os.path.join("../dataset2/res_still/train/table", "*.jpg"))
val_pot   = glob(os.path.join("../dataset2/res_still/valid/potato", "*.jpg"))
val_cat   = glob(os.path.join("../dataset2/res_still/valid/catfood", "*.jpg"))
val_tab   = glob(os.path.join("../dataset2/res_still/valid/table", "*.jpg"))
test_pot  = glob(os.path.join("../dataset2/res_still/test/potato", "*.jpg"))
test_cat  = glob(os.path.join("../dataset2/res_still/test/catfood", "*.jpg"))
test_tab  = glob(os.path.join("../dataset2/res_still/test/table", "*.jpg"))

X_train, y_train = ii.images_to_numpy(train_pot, train_cat, train_tab)
X_val, y_val = ii.images_to_numpy(val_pot, val_cat, val_tab)
X_test, y_test = ii.images_to_numpy(test_pot, test_cat, test_tab)

batch_size = 32
num_classes = 3

# Making datagen for training with normilization, center, and dataugmentation
train_datagen = ImageDataGenerator(
        featurewise_center=True,
        #featurewise_std_normalization=True
        rescale=1./255,
        rotation_range=10,
        vertical_flip=True,
        horizontal_flip=True,
        brightness_range=(0.65, 1.35)
)

# Making datagen for validation and test with normilization
valid_datagen = ImageDataGenerator(
        featurewise_center=True, 
        #featurewise_std_normalization=True
        rescale=1./255
)


X_train = X_train.astype(float)
X_val = X_val.astype(float)
X_test = X_test.astype(float)

y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

# Required for featurewise normalization
train_datagen.fit(X_train/255.)
valid_datagen.fit(X_train/255.)

train_generator = train_datagen.flow(X_train,
                                     y_train,
                                     batch_size=batch_size, shuffle=True)

validation_generator = valid_datagen.flow(X_val,
                                     y_val,
                                     batch_size=batch_size, shuffle=True)

test_generator = valid_datagen.flow(X_test,
                                     y_test,
                                     batch_size=80, shuffle=True)


## Optimized model single run

In [ ]:
model_optimized = Sequential()
model_optimized.add(Flatten(input_shape=(224,224,3)))
model_optimized.add(Dense(256))
model_optimized.add(Activation('relu'))
model_optimized.add(Dense(3))
model_optimized.add(Activation('softmax'))
model_optimized.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

model_optimized.summary()

In [ ]:
history_optimized = model_optimized.fit_generator(
                    train_generator,
                    steps_per_epoch=573 // batch_size,
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps=160 // batch_size)


In [ ]:
plot_history(history_optimized, save=True, path='nets/feedforward/', name='plot_full')

In [ ]:
imgs, labels = next(test_generator)

#ynew = model.predict(imgs)
#results[(lr, lrd, rs, hl)] = (y_train_acc, y_pred_acc)

history_optimized.evaluate(imgs,labels, batch_size=80)

In [ ]:
imgs, labels = next(validation_generator)
history_optimized.predict(imgs)

## Hyperparameter optimization

In [ ]:
# Only work with tensorflow=1.8 and keras=2.1.6 because of the memory allocation

# Hyperparameters
learning_rates = [1e-4, 1e-5];
hidden_size = [128, 256, 512];
history = []
result = None
results = []

for lr in learning_rates:
    for hs in hidden_size:
        # GPU memory allocation
        cfg = K.tf.ConfigProto() # Create setup file for keras tensorflow
        cfg.gpu_options.allow_growth = True # Setting setup file with dynamically adding gpu memory
        K.set_session(K.tf.Session(config=cfg)) # Setting session to use setup file
        sess = tf.Session() 
        K.set_session(sess)# set this TensorFlow session as the default session for Keras.
        
        # Defining the model:
        model = Sequential()
        model.add(Flatten(input_shape=(224,224,3)))
        model.add(Dense(hs))
        model.add(Activation('relu'))
        model.add(Dense(3))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(lr=lr),
                      metrics=['accuracy'])
        
        
        # Running the model:
        temp = (model.fit_generator(
            train_generator,
            steps_per_epoch=573 // batch_size,
            epochs=100,
            validation_data=validation_generator,
            validation_steps=160 // batch_size))
        
        # Saving the results
        history.append(temp)
        train_acc = temp.history['acc'].pop()
        val_acc = temp.history['val_acc'].pop()
        result = (lr, hs, train_acc, val_acc)
        results.append(result)
        K.clear_session() # Clearing the session to clear out gpu memory
        

#### Printing all results and plotting only the best validation accuracy

In [ ]:
best_valacc_indices = []
highest_valacc = 0
for index, r in enumerate(results):
    print('lr %e hs %i train accuracy: %f val accuracy: %f' % r) 
    if highest_valacc < r[3]:
        best_valacc_indices = [] # Empty the list because of better value
        best_valacc_indices.append(index)
        highest_valacc = r[3]
    elif highest_valacc == r[3]:
        best_valacc_indices.append(index)

In [ ]:
for indices in best_valacc_indices:
    print("______________________________________________________________________")
    print('lr %e hs %i train accuracy: %f val accuracy: %f' % results[indices])         
    plot_history(history[indices], mean_N=1)

**Based on graph and results above of the training and validation accuracy the following hyperparameters is found to be the optimal:** 
```
Learning rate       = 1e-5
Hidden size         = 256
```
**Which gives the following results for 100 epoch for 1 training run:**
```
Train accuracy      = 0.987500
Validation accuracy = 0.983364
```

## Mean and standard deviation for optimal hyperparameters

In [24]:
history_mean_std = []
iterations = 30

# Defining the model:
model_mean_std = Sequential()
model_mean_std.add(Flatten(input_shape=(224,224,3)))
model_mean_std.add(Dense(256))
model_mean_std.add(Activation('relu'))
model_mean_std.add(Dense(3))
model_mean_std.add(Activation('softmax'))
model_mean_std.compile(loss='categorical_crossentropy',
                          optimizer=Adam(lr=1e-5),
                          metrics=['accuracy'])

for it in range(iterations):
        # Training the model:
        temp = (model_mean_std.fit_generator(
            train_generator,
            steps_per_epoch=573 // batch_size,
            epochs=30,
            validation_data=validation_generator,
            validation_steps=160 // batch_size))
        
        # Saving the results
        history_mean_std.append(temp)

Epoch 1/30
17/17 [==============================] - 5s 308ms/step - loss: 1.1183 - acc: 0.4017 - val_loss: 1.0259 - val_acc: 0.3812
Epoch 2/30
17/17 [==============================] - 5s 275ms/step - loss: 0.9543 - acc: 0.5551 - val_loss: 0.9265 - val_acc: 0.7812
Epoch 3/30
17/17 [==============================] - 5s 280ms/step - loss: 0.8905 - acc: 0.5945 - val_loss: 0.8609 - val_acc: 0.6250
Epoch 4/30
17/17 [==============================] - 5s 286ms/step - loss: 0.8178 - acc: 0.6471 - val_loss: 0.7654 - val_acc: 0.8625
Epoch 5/30
17/17 [==============================] - 5s 279ms/step - loss: 0.7525 - acc: 0.7302 - val_loss: 0.8268 - val_acc: 0.6438
Epoch 6/30
17/17 [==============================] - 5s 284ms/step - loss: 0.7209 - acc: 0.7518 - val_loss: 0.7237 - val_acc: 0.8250
Epoch 7/30
17/17 [==============================] - 5s 281ms/step - loss: 0.6669 - acc: 0.7694 - val_loss: 0.7261 - val_acc: 0.8063
Epoch 8/30
17/17 [==============================] - 5s 289ms/step - loss: 0.

In [25]:
temp = None
temp_his = []
model_mean = None
model_std = None

for h in history_mean_std:
    temp = (h.history['loss'], h.history['acc'], h.history['val_loss'], h.history['val_acc'])
    temp_his.append(temp)

temp_his = np.asarray(temp_his) # Making it to numpy array
model_mean = np.mean(temp_his, axis=0) # Calculating mean for all points
model_std = np.std(temp_his, axis=0) # Standard deviation for all points

# Brug plt.plot for at vise plots

# Printing the last value of loss, acc, vall_loss and vall_acc
string_val = ["Train Loss:     ", "Train Accuracy: ", "Valid Loss:     ", "Valid Accuracy: "]

print("----------------Mean------------------")
for index, i in enumerate(model_mean):
    print(string_val[index] + str(i[-1]))
    
print("---------Standard Deviation-----------")
for index, i in enumerate(model_std):
    print(string_val[index] + str(i[-1]))
     

----------------Mean------------------
Train Loss:     0.19606438312892993
Train Accuracy: 0.9602587796513562
Valid Loss:     0.2105408251285553
Valid Accuracy: 0.94375
---------Standard Deviation-----------
Train Loss:     0.07001356870734758
Train Accuracy: 0.019408502166681052
Valid Loss:     0.07460179030895234
Valid Accuracy: 0.025000000000000022
